### **1. Import relevant packages**

In [1]:
#!pip install openai qdrant-client fastembed datasets langchain python-dotenv 
!pip install deepeval

/Users/atitaarora/.zshenv:2: command too long: /Users/atitaarora/qdrant/workspace/qdrant-rag-eval/workshop-rag-eval-oxford-llm2024/oxford-rag-eval/bin:/opt/homebrew/bin:/opt/homebrew/sbin:/usr/local/bin:/System/Cryptexes/App/usr/bin:/usr/bin:/bin:/usr/sbin:/sbin:/var/run/com.apple.security.cryptexd/codex.system/bootstrap/usr/local/bin:/var/run/com.apple.security.cryptexd/codex.system/bootstrap/usr/bin:/var/run/com.apple.security.cryptexd/codex.system/bootstrap/usr/appleinternal/bin:/Library/Apple/usr/bin:/Users/atitaarora/.cargo/bin:/Applications/iTerm.app/Contents/Resources/utilities=/Users/atitaarora/qdrant/workspace/qdrant-rag-eval/workshop-rag-eval-oxford-llm2024/oxford-rag-eval/bin:/opt/homebrew/bin:/opt/homebrew/sbin:/usr/local/bin:/System/Cryptexes/App/usr/bin:/usr/bin:/bin:/usr/sbin:/sbin:/var/run/com.apple.security.cryptexd/codex.system/bootstrap/usr/local/bin:/var/run/com.apple.security.cryptexd/codex.system/bootstrap/usr/bin:/var/run/com.apple.security.cryptexd/codex.system/

In [ ]:
# Get a FREE forever cluster at https://cloud.qdrant.io/
# More info: https://qdrant.tech/documentation/cloud/create-cluster/
QDRANT_URL = "<URL>"
QDRANT_API_KEY = "<API_KEY>"
# Get your key from https://platform.openai.com/api-keys
OPENAI_API_KEY = "<API_KEY>"

In [ ]:
import os
import json
import openai
from tqdm.notebook import tqdm
from datasets import load_dataset
from qdrant_client import QdrantClient

In [ ]:
load_dotenv()

In [ ]:
dataset = load_dataset("atitaarora/qdrant_doc", split="train")


In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document as LangchainDocument

langchain_docs = [
    LangchainDocument(page_content=doc["text"], metadata={"source": doc["source"]})
    for doc in tqdm(dataset)
]


In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=512,
    chunk_overlap=50,
    add_start_index=True,
    separators=["\n\n", "\n", ".", " ", ""],
)

docs_processed = []
for doc in langchain_docs:
    docs_processed += text_splitter.split_documents([doc])
len(docs_processed)

9267

In [ ]:
client = QdrantClient(
    url=QDRANT_URL,
    api_key=QDRANT_API_KEY
)

In [ ]:
COLLECTION_NAME = "qdrant-deepeval"

In [ ]:
docs_contents = []
docs_metadatas = []

for doc in docs_processed:
    if hasattr(doc, 'page_content') and hasattr(doc, 'metadata'):
        docs_contents.append(doc.page_content)
        docs_metadatas.append(doc.metadata)
    else:
        # Handle the case where attributes are missing
        print("Warning: Some documents do not have 'page_content' or 'metadata' attributes.")

len(docs_contents)
len(docs_metadatas)

content :  9267
metadata :  9267


In [ ]:
client.add(collection_name=COLLECTION_NAME, metadata=docs_metadatas, documents=docs_contents)

In [ ]:
client.count(collection_name=COLLECTION_NAME).count

In [ ]:
from openai import OpenAI
openai_client = OpenAI(api_key=OPENAI_API_KEY)

In [ ]:
def query_with_context(query,limit):

    ## Fetch context from Qdrant
    search_result = client.query(collection_name=COLLECTION_NAME, query_text=query, limit=limit)

    contexts = [
        "document: "+r.document+",source: "+r.metadata['source'] for r in search_result
    ]
    prompt_start = (
        """ You're assisting a user who has a question based on the documentation.
        Your goal is to provide a clear and concise response that addresses their query while referencing relevant information
        from the documentation.
        Remember to:
        Understand the user's question thoroughly.
        If the user's query is general (e.g., "hi," "good morning"),
        greet them normally and avoid using the context from the documentation.
        If the user's query is specific and related to the documentation, locate and extract the pertinent information.
        Craft a response that directly addresses the user's query and provides accurate information
        referring the relevant source and page from the 'source' field of fetched context from the documentation to support your answer.
        Use a friendly and professional tone in your response.
        If you cannot find the answer in the provided context, do not pretend to know it.
        Instead, respond with "I don't know".

        Context:\n"""
    )

    prompt_end = (
        f"\n\nQuestion: {query}\nAnswer:"
    )

    prompt = (
        prompt_start + "\n\n---\n\n".join(contexts) +
        prompt_end
    )

    res = openai_client.completions.create(
        model="gpt-3.5-turbo-instruct",
        prompt=prompt,
        temperature=0,
        max_tokens=636,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0,
        stop=None
    )

    return (contexts , res.choices[0].text)

In [ ]:
question = "what is quantization?"

In [ ]:
context , rag_response = query_with_context(question, 3)
print(rag_response)

In [ ]:
qdrant_qna_dataset = load_dataset("atitaarora/qdrant_doc_qna", split="train")

EVAL_SIZE = 10
RETRIEVAL_SIZE = 3

In [ ]:
from deepeval.metrics import (
    AnswerRelevancyMetric,
    FaithfulnessMetric,
    ContextualPrecisionMetric,
    ContextualRecallMetric,
    ContextualRelevancyMetric
)

In [ ]:
from datasets import Dataset
from deepeval.test_case import LLMTestCase


def create_deepeval_dataset(dataset, eval_size,retrieval_window_size):
    test_cases = []
    for i in range(eval_size):
        entry = dataset[i]
        question = entry["question"]
        answer = entry["answer"]
        context, rag_response = query_with_context(question, retrieval_window_size)
        test_case = LLMTestCase(
            input=question,
            actual_output=rag_response,
            expected_output=answer,
            retrieval_context=context,
        )
        test_cases.append(test_case)
    return test_cases



In [ ]:
test_cases = create_deepeval_dataset(qdrant_qna_dataset, EVAL_SIZE, RETRIEVAL_SIZE)

In [51]:
!deepeval login

Welcome to DeepEval!
Login and grab your API key here: ]8;id=816109;https://app.confident-ai.com\https://app.confident-ai.com]8;;\ 
Paste your API Key: zZEY2IKxuVegV0Ob1VySEHM3VNh1JC0xcUc1BNehKTM=
Congratulations! Login successful 🙌 
If you are new to DeepEval, follow our quickstart tutorial here: 
]8;id=591976;https://docs.confident-ai.com/docs/getting-started\https://docs.confident-ai.com/docs/getting-started]8;;\


In [ ]:
from deepeval import evaluate

evaluate(
    test_cases=test_cases,
    metrics=[
        AnswerRelevancyMetric(),
        FaithfulnessMetric(),
        ContextualPrecisionMetric(),
        ContextualRecallMetric(),
        ContextualRelevancyMetric(),
    ],
)